# CPP: Identification of physicochemical signatures


In [ ]:
import aaanalysis as aa
aa.options["verbose"] = False

More details on the ``CPP`` and ``CPPPlot`` classes are given in the [Feature Engineering API](https://aaanalysis.readthedocs.io/en/latest/api.html#feature-engineering). To delve deeper into the feature concept, see the [CPP Usage Principles](https://aaanalysis.readthedocs.io/en/latest/index/usage_principles/feature_identification.html) section.